# Lab 01 - Histograms manipulations and Filtering

Computer Vision 2023-24 (M. Caligiuri, P. Zanuttigh, F. Lincetto)

In this lab you will explore two different topics:

1. Histogram-based processing:
    - Histogram equalization,
    - Histogram matching;
2. Filtering for denoising:
    - Gaussian filter,
    - Median filter,
    - Bilateral filter.

---

## Import the necessary libraries

In [ ]:
# Import the necessary packages
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

---

## Histogram equalization

In this section you will have to perform the histogram equalization of the image `./data/barbecue.jpg`. Do that using first the **BGR** color space and then the **LAB** color space. You will see the difference in the results.

### Auxiliary functions

Write here all the auxiliary functions you will need.
More precisely you have to implement the following functions:
- `calc_hist` -> given in input an image it will return a list containing the histogram of each channel,
- `show_histogram` -> given in input the list of all the histograms of an image it will return an image containing the plot of the histograms of each channel side by side (use matplotlib to generate the plot than convert them to image using `cv2`),

  ![hists_example](data/hists.png)
- `img_hist` -> given in input an image and the correspondent list of histograms it will return the image with the histograms plotted on top of it (call `show_histogram`).

  ![img_hist_example](data/img_hists.png)

In [ ]:
# Define a function to compute the histogram of the image (channel by channel)
def calc_hists(img: np.ndarray) -> list:
    """
    Calculates the histogram of the image (channel by channel).

    Args:
        img (numpy.ndarray): image to calculate the histogram
    
    Returns:
        list: list of histograms
    """

    # Add your code here
    
    # Separate the image channels
    channels = cv.split(img)

    # Calculate histograms for each channel
    hist_list = [cv.calcHist([channel], [0], None, [256], [0, 256]) for channel in channels]

    return hist_list

In [ ]:
# Define a function to plot together the three histograms of the image
# (plot only one if it is grayscale)
def show_histogram(hists: list) -> np.ndarray:
    """
    Shows the histogram of the image.
    
    Args:
        hists (list): list of histograms to plot

    Returns:
        img (numpy.ndarray): image containing the histograms
     """   
   

    # Add your code here to generate the matplotlib figurecontaining the histograms
    
    fig, ax = plt.subplots()

    # Plot each histogram in a different color
    colors = ['b', 'g', 'r']
    for i, hist in enumerate(hists):
        ax.plot(hist, color=colors[i], label=f'Channel {i+1}')

    # Add labels and legend
    ax.set_title('Histogram')
    ax.set_xlabel('Pixel Value')
    ax.set_ylabel('Frequency')
    ax.legend()


    # Convert figure to canvas
    canvas = plt.get_current_fig_manager().canvas
    plt.close()

    # Render the canvas
    canvas.draw()

    # Convert canvas to image
    img = np.frombuffer(canvas.buffer_rgba(), dtype='uint8')
    img = img.reshape(canvas.get_width_height()[::-1] + (4,))

    # Convert image to cv2 format
    return cv.cvtColor(img, cv.COLOR_RGB2BGR)

In [ ]:
# Define a function to fuse the histogram image with the original one
def img_hist(img: np.ndarray, hists: list[np.ndarray], scaling_factor: float = 0.3) -> np.ndarray:
    """
    Fuse the image with the histogram.
    
    Args:
        img (np.ndarray): image
        hist (list(np.ndarray)): histograms of the image
        
    Returns:
        img (numpy.ndarray): image containing the original image with the histograms on top
    """
    
    # Add your code here
    hist_img = show_histogram(hists)

    # Ensure the histogram image has the same height as the original image
    hist_img_resized = cv.resize(hist_img, (hist_img.shape[1], img.shape[0]))

    # Blend the original image and the histogram image
    result_img = cv.addWeighted(img, 1.0 - scaling_factor, hist_img_resized, scaling_factor, 0)

    return result_img



### Equalize the image in the BGR space

Equalize the image (`./data/barbecue.jpg`) in the BGR space and show the result.

In [ ]:
# Load image and and compute its histogram

# Add your code here
# Load image
img_path = './data/barbecue.jpg'
img = cv.imread(img_path)

# Calculate histograms in BGR color space
bgr_hist_list = calc_hists(img)

In [ ]:
# Show the image with its histogram before the equalization

# Add your code here

# Display the image using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
plt.title('Original Image')
plt.axis('off')
plt.show()

# Display the histograms using matplotlib
plt.figure(figsize=(15, 4))

plt.subplot(1, 3, 1)
plt.plot(bgr_hist_list[0], color='b', label='Channel 1')
plt.title('Blue Channel Histogram')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(bgr_hist_list[1], color='g', label='Channel 2')
plt.title('Green Channel Histogram')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(bgr_hist_list[2], color='r', label='Channel 3')
plt.title('Red Channel Histogram')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.legend()

plt.show()

In [ ]:
# Equalize the histogram of the image

# Add your code here
# Convert BGR image to HSV
img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

# Equalize the histogram of the V (Value/Luminance) channel
img_hsv[:,:,2] = cv.equalizeHist(img_hsv[:,:,2])

# Convert the image back to BGR
equalized_img = cv.cvtColor(img_hsv, cv.COLOR_HSV2BGR)

# Show the image and the relative histogram (in the same window)

# Add your code here

plt.figure(figsize=(15, 6))

# Display the equalized image
plt.subplot(1, 2, 1)
plt.imshow(cv.cvtColor(equalized_img, cv.COLOR_BGR2RGB))
plt.title('Equalized Image')
plt.axis('off')

# Calculate histograms for the equalized image
equalized_hist_list = calc_hists(equalized_img)

# Display the histograms for the equalized image
plt.subplot(1, 2, 2)
plt.plot(equalized_hist_list[0], color='b', label='Channel 1')
plt.plot(equalized_hist_list[1], color='g', label='Channel 2')
plt.plot(equalized_hist_list[2], color='r', label='Channel 3')
plt.title('Histograms After Equalization')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.legend()

plt.show()

### Equalize the image in the LAB space

Equalize the image (`./data/barbecue.jpg`) in the LAB space and show the result.

In [ ]:
# Load the image and convert it to LAB colorspace

# Add your code here

# Load image
img_path = './data/barbecue.jpg'
img = cv.imread(img_path)

# Convert BGR image to LAB
img_lab = cv.cvtColor(img, cv.COLOR_BGR2LAB)


In [ ]:
# Equalize the histogram of the L channel

# Add your code here

# Convert BGR image to LAB
img_lab = cv.cvtColor(img, cv.COLOR_BGR2LAB)

# Split LAB image into L, A, and B channels
l_channel, a_channel, b_channel = cv.split(img_lab)

# Equalize the histogram of the L channel
l_channel_equalized = cv.equalizeHist(l_channel)

# Merge the equalized L channel with the original A and B channels
img_lab_equalized = cv.merge([l_channel_equalized, a_channel, b_channel])

# Convert the image back to BGR
img_equalized = cv.cvtColor(img_lab_equalized, cv.COLOR_LAB2BGR)

# Convert the image back to BGR colorspace

# Add your code here

img_equalized_bgr = cv.cvtColor(img_lab_equalized, cv.COLOR_LAB2BGR)

# Extract the histograms for each channel

# Add your code here

# Calculate histograms for each channel in the equalized image
hist_equalized_b, edges_equalized_b = np.histogram(img_equalized_bgr[:,:,0].ravel(), bins=256, range=[0,256])
hist_equalized_g, edges_equalized_g = np.histogram(img_equalized_bgr[:,:,1].ravel(), bins=256, range=[0,256])
hist_equalized_r, edges_equalized_r = np.histogram(img_equalized_bgr[:,:,2].ravel(), bins=256, range=[0,256])

# Optionally normalize histograms if needed
hist_equalized_b = hist_equalized_b / hist_equalized_b.sum()
hist_equalized_g = hist_equalized_g / hist_equalized_g.sum()
hist_equalized_r = hist_equalized_r / hist_equalized_r.sum()

In [ ]:
# Show the image and its relative histogram

# Add your code here

# Display the images and histograms using matplotlib
plt.figure(figsize=(15, 6))

# Display the equalized image
plt.subplot(1, 2, 1)
plt.imshow(cv.cvtColor(img_equalized_bgr, cv.COLOR_BGR2RGB))
plt.title('Equalized Image')
plt.axis('off')

# Calculate histograms for each channel in the equalized image
hist_equalized_b, edges_equalized_b = np.histogram(img_equalized_bgr[:,:,0].ravel(), bins=256, range=[0,256])
hist_equalized_g, edges_equalized_g = np.histogram(img_equalized_bgr[:,:,1].ravel(), bins=256, range=[0,256])
hist_equalized_r, edges_equalized_r = np.histogram(img_equalized_bgr[:,:,2].ravel(), bins=256, range=[0,256])

# Optionally normalize histograms if needed
hist_equalized_b = hist_equalized_b / hist_equalized_b.sum()
hist_equalized_g = hist_equalized_g / hist_equalized_g.sum()
hist_equalized_r = hist_equalized_r / hist_equalized_r.sum()

# Display the histograms
plt.subplot(1, 2, 2)
plt.plot(hist_equalized_b, color='b', label='Channel 1')
plt.plot(hist_equalized_g, color='g', label='Channel 2')
plt.plot(hist_equalized_r, color='r', label='Channel 3')
plt.title('Histograms After Equalization')
plt.xlabel('Pixel Value')
plt.ylabel('Normalized Frequency')
plt.legend()

plt.show()


---

## Histogram matching

In this section you will have to perform the histogram matching of the image `./data/panorama_left.jpg` to the image `./data/panorama_right.jpg`. Do that in the **BGR** color space.

### Histogram matching function

The function to perform the histogram matching is provided here. The function takes in input the image to be matched and the image to match and will return the matched image.

In [ ]:
#  function to perform the histogram matching
def hist_match(source: np.ndarray, reference: np.ndarray) -> np.ndarray:
    """
    Adjust the pixel values of a color image such that its histogram
    matches that of a target one.

    Args:
        source (numpy.ndarray): Image to transform; the histogram is computed over the flattened array
        reference (numpy.ndarray): Template image; can have different dimensions from source
    Returns:
        numpy.ndarray: The transformed output image
    """
    
     # Assert that the images have the same number of channels (grayscale or RGB) and the same dimensions
    assert source.shape[2] == reference.shape[2], "Images must have the same number of channels"
    assert source.shape[:2] == reference.shape[:2], "Images must have the same dimensions"

    # Compute the source image's histogram and CDF
    src_hists = [np.histogram(source[..., i].flatten(), 256, [0,256])[0] for i in range(source.shape[2])]
    src_cdfs = [hist.cumsum() for hist in src_hists]
    src_cdfs_normalized = [cdf / float(cdf.max()) for cdf in src_cdfs]
 
    # Compute the reference image's histogram and CDF
    ref_hists = [np.histogram(reference[..., i].flatten(), 256, [0,256])[0] for i in range(reference.shape[2])]
    ref_cdfs = [hist.cumsum() for hist in ref_hists]
    ref_cdfs_normalized = [cdf / float(cdf.max()) for cdf in ref_cdfs]

    # Create a lookup table to map pixel values from the source to the reference
    lookup_tables = [np.zeros(256) for _ in range(source.shape[2])]
    lookup_values = [0] * source.shape[2]
    for index in range(len(lookup_tables)):
        for src_pixel_val in range(len(src_cdfs_normalized[index])):
            lookup_values[index]
            for ref_pixel_val in range(len(ref_cdfs_normalized[index])):
                if ref_cdfs_normalized[index][ref_pixel_val] >= src_cdfs_normalized[index][src_pixel_val]:
                    lookup_values[index] = ref_pixel_val
                    break
            lookup_tables[index][src_pixel_val] = lookup_values[index]

    # Apply the lookup table to the source image
    matched = np.stack([cv.LUT(source[..., i], lookup_tables[i]).astype(np.uint8) for i in range(len(lookup_tables))], axis=-1)

    return matched

### Match the images

In [ ]:
# Load the two iamges

# Add your code here

image_left_path = './data/panorama_left.jpg'
image_right_path = './data/panorama_right.jpg'

image_left = cv.imread(image_left_path)
image_right = cv.imread(image_right_path)


In [ ]:
# Visualize the two images with the relative histograms
# before the histogram matching

# Add your code here

plt.figure(figsize=(20, 16))

# Panorama Left
plt.subplot(2, 2, 1)
plt.imshow(cv.cvtColor(image_left, cv.COLOR_BGR2RGB))
plt.title('Panorama Left')
plt.axis('off')

plt.subplot(2, 2, 2)
hist_img_left = show_histogram(calc_hists(image_left))
plt.imshow(hist_img_left)
plt.title('Histogram for Panorama Left')
plt.axis('off')

# Panorama Right
plt.subplot(2, 2, 3)
plt.imshow(cv.cvtColor(image_right, cv.COLOR_BGR2RGB))
plt.title('Panorama Right')
plt.axis('off')

plt.subplot(2, 2, 4)
hist_img_right = show_histogram(calc_hists(image_right))
plt.imshow(hist_img_right)
plt.title('Histogram for Panorama Right')
plt.axis('off')

plt.show()

In [ ]:
# Match the histograms of the two images

# Add your code here

matched_image = hist_match(image_left, image_right)


# Visualize the matched images along with their histograms
plt.figure(figsize=(20, 16))

# Matched Image
plt.subplot(2, 2, 1)
plt.imshow(cv.cvtColor(matched_image, cv.COLOR_BGR2RGB))
plt.title('Matched Image')
plt.axis('off')

# Matched Histogram for Left Image 
plt.subplot(2, 2, 2)
hist_img_matched = show_histogram(calc_hists(matched_image))
plt.imshow(hist_img_matched)
plt.title('Matched Histogram for Panorama')
plt.axis('off')

plt.show()

---

## Image denoising

In this section you will have to perform the denoising of the image ./data/overexposed.jpg. Do that using the following filters:

    Gaussian filter,
    Median filter,
    Bilateral filter.

For each filter you will have to try different values for the parameters in order to see the effect of each parameter on the result. To do that in an interactive way it is required that you implement the cv.trackbar function. In this way you will be able to change the parameters of the filters and to see the result in real time.

In order to build a better code you have to implement each filter as a class (child of a main filter class called Filter). The structure to do this is already provided in this notebook

### Define the filters classes

In [ ]:
# Define the main Filter class with the common methods
# (this class will be used as a base class for the other filters)
class Filter:
    def __init__(self, size: int):
        # Constructor of the class
        if size % 2 == 0:
            size += 1
        self.set_filter_size(size)
    
    def set_filter_size(self, size: int) -> None:
        # Method to set the filter size
        if size % 2 == 0:
            size += 1
        self.filter_size = size
    
    def get_filter_size(self) -> int:
        # Method to get the filter size
        return self.filter_size

    def __call__(self, image: np.ndarray) -> None:
        # Method to perform the filtering
        # (it must be implemented in the derived classes)
        raise NotImplementedError

In [ ]:
# Define the class for the gaussian filter
class GaussianFilter(Filter):
    def __init__(self, size: int, sigma_g: float):
        super().__init__(size)
        self.sigma = sigma_g

    def set_sigma(self, s: float) -> None:
        # Method to set the sigma value

        # Add your code here (remove the pass statement)
        self.sigma = s

    def get_sigma(self) -> float:
        # Method to get the sigma value
        
        # Add your code here (remove the pass statement)
        return self.sigma
    
    def __call__(self, image: np.ndarray, size: int, sigma: float) -> None:
        # Method to call the filter (it is called when the object is called)
        # This function will be called by the trackbar object and will update the imamge to show
        
        # Add your code here (remove the pass statement)
        filtered_image = cv.GaussianBlur(image, self.get_filter_size(), self.get_sigma(), 0, cv.BORDER_DEFAULT )        
        return filtered_image


# Define the class for the median filter
class MedianFilter(Filter):
    def __init__(self, size: int):
        super().__init__(size)

    def __call__(self, image: np.ndarray, size: int) -> None:
        # Method to call the filter (it is called when the object is called)
        # This function will be called by the trackbar object and will update the image to show
        
        # Add your code here (remove the pass statement)
        filtered_image = cv.medianBlur(image, self.get_filter_size())     
        return filtered_image


# Define the class for the bilateral filter
class BilateralFilter(Filter):
    def __init__(self, size: int, sigma_s: float, sigma_r: float):
        super().__init__(size)
        self.sigma_space = sigma_s
        self.sigma_range = sigma_r

    def set_sigma_range(self, sr: float) -> None:
        # Method to set the sigma range value

        # Add your code here (remove the pass statement)
        self.sigma_range = sr

    def get_sigma_range(self) -> float:
        # Method to get the sigma range value
        
        # Add your code here (remove the pass statement)
        return self.sigma_range

    def set_sigma_space(self, ss: float) -> None:
        # Method to set the sigma space value

        # Add your code here (remove the pass statement)
        self.sigma_space = ss

    def get_sigma_space(self) -> float:
        # Method to get the sigma space value
        
        # Add your code here (remove the pass statement)
        return self.sigma_space
    
    def __call__(self, image: np.ndarray, sigma_s: float, sigma_r: float) -> None:
        # Method to call the filter (it is called when the object is called)
        # This function will be called by the trackbar object and will update the imamge to show
        
        # Add your code here (remove the pass statement)
        filtered_image = cv.bilateralFilter(image, self.get_filter_size(), self.get_sigma_range(), self.get_sigma_space())        
        return filtered_image

### Load the image and initialize the filters objects

In [ ]:
# Load and show the image to filter

# Add your code here
image_path = './data/overexposed.jpg'
image = cv.imread(image_path)

# Convert BGR to RGB for displaying with matplotlib
image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)

# Display the image
plt.imshow(image_rgb)
plt.title('Original Image')
plt.axis('off')
plt.show()

In [ ]:
# Initialize the filters

# Add your code here
gaussian_filter = GaussianFilter(size=5, sigma_g=1.0)
median_filter = MedianFilter(size=5)
bilateral_filter = BilateralFilter(size=5, sigma_s=75, sigma_r=75)


### Test the filters

In [ ]:
# MEDIAN BLUR FILTER

# Create a window for the trackbars
cv.namedWindow('Median Blur Filter Trackbars')


# Callback function for the trackbar
def trackbar_callback(value):
    print(f"Trackbar value: {value}")

# Create the trackbar
cv.createTrackbar("Kernel Size", "Median Blur Filter Trackbars", 1, 30, trackbar_callback)

for k_size in range(1,31,2):

    # Apply the Median filter to the image with the current kernel size
    original_image = image
    filtered_image = cv.medianBlur(image, k_size)
    
    # Display the original and filtered images using matplotlib
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(original_image)
    plt.title('Original Image')
    
    plt.subplot(1, 2, 2)
    plt.imshow(filtered_image)
    plt.title(f'Filtered Image (Kernel Size: {k_size})')
    
    plt.pause(0.1)  
    
# Destroy the window
cv.destroyAllWindows()

In [ ]:
# GAUSSIAN BLUR FILTER


           
# Create a window for the trackbars
cv.namedWindow('Gaussian Blur Filter Trackbars')


# Callback function for the trackbar
def trackbar_callback(value):
    print(f"Trackbar value: {value}")

# Create the trackbar
cv.createTrackbar("Kernel Size", "Gaussian Blur Filter Trackbars", 1, 30, trackbar_callback)

# Iterate over different sigma values
for sigma_g in range(10, 100, 20): 
    # Iterate over different kernel sizes
    for kernel_size in range(1, 31, 2):  # Ensure the kernel size is always odd
        # Apply the Gaussian filter to the image with the current kernel size and sigma value
        filtered_image = cv.GaussianBlur(image, (kernel_size, kernel_size), sigma_g, 0, cv.BORDER_DEFAULT)
        # Display the original and filtered images using matplotlib
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(image)
        plt.title('Original Image')

        plt.subplot(1, 2, 2)
        plt.imshow(filtered_image)
        plt.title(f'Filtered Image (Kernel Size: {kernel_size}, Sigma: {sigma_g})')

        plt.pause(0.1)

# Destroy the window
cv.destroyAllWindows()


In [ ]:
# BILATERAL FILTER


# Create a window for the trackbars

# Add your code here
cv.namedWindow('Bilateral Blur Filter Trackbars')

# Callback function for the trackbar
def trackbar_callback(value):
    print(f"Trackbar value: {value}")

# Create the trackbar
cv.createTrackbar("Kernel Size", "Bilateral Blur Filter Trackbars", 1, 30, trackbar_callback)

# Show the filtered image with a slider to change the kernel size

# Add your code here
# Define fixed parameters for the bilateral filter
for sigma_r in range(10, 100, 20):  
    for sigma_s in range(10, 100, 20):  
        # Iterate over different kernel sizes
        for kernel_size in range(1, 31):
            # Apply the Bilateral filter to the image with the current parameters
            filtered_image = cv.bilateralFilter(image, kernel_size, sigma_r, sigma_s)

            # Display the original and filtered images using matplotlib
            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(image)
            plt.title('Original Image')

            plt.subplot(1, 2, 2)
            plt.imshow(filtered_image)
            plt.title(f'Filtered Image (Kernel Size: {kernel_size}, σr: {sigma_r}, σs: {sigma_s})')

            plt.pause(0.1)  
            
# Destroy the window
cv.destroyAllWindows()
